## 结构化输出 (Structured Output)

在这个notebook中，我们将演示如何让LLM返回结构化的数据格式，而不是纯文本。我们将使用与chatbot相同的例子：求Normal分布的最大似然估计(MLE)。

### 为什么需要结构化输出？
- 便于程序处理和解析
- 减少格式错误
- 提高数据的可靠性和一致性
- 便于与其他系统集成

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
import dotenv

dotenv.load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [2]:
import json
from langchain_core.messages import HumanMessage, SystemMessage

system_prompt = """
你是一个数学助手。请用以下JSON格式回答问题：
{
  "answer": "答案",
  "explanation": "推导过程"
}

请确保返回的是有效的JSON格式，不要包含任何额外的文本。
"""

user_question = "Normal分布的最大似然估计(MLE)公式是什么？"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question)
]

response = model.invoke(messages)
print("原始回复:")
print(response.content)
print("\n" + "="*50 + "\n")

# 手动解析JSON
try:
    # 尝试解析JSON
    content = str(response.content)
    parsed_result = json.loads(content)
    print("解析成功！")
    print("答案:")
    print(parsed_result["answer"])
    print("\n解释:")
    print(parsed_result["explanation"])
except json.JSONDecodeError as e:
    print(f"JSON解析失败: {e}")
    print("原始内容:")
    print(response.content)


原始回复:
```json
{
  "answer": "对于一组独立同分布的样本 $x_1, x_2, \ldots, x_n$ 来源于正态分布 $N(\mu, \sigma^2)$，其最大似然估计（MLE）公式如下：\n\n1.  **均值 (Mean) 的 MLE：**\n    $\\hat{\\mu}_{MLE} = \\bar{x} = \\frac{1}{n} \\sum_{i=1}^n x_i$\n\n2.  **方差 (Variance) 的 MLE：**\n    $\\hat{\\sigma}^2_{MLE} = \\frac{1}{n} \\sum_{i=1}^n (x_i - \\bar{x})^2$",
  "explanation": "最大似然估计（MLE）的目标是找到使观测数据出现概率最大的参数值。对于正态分布，我们有其概率密度函数（PDF）：\n$f(x|\\mu, \\sigma^2) = \\frac{1}{\\sqrt{2\\pi\\sigma^2}} e^{-\\frac{(x-\\mu)^2}{2\\sigma^2}}$\n\n1.  **构建似然函数 (Likelihood Function)：**\n    对于 $n$ 个独立同分布的样本 $x_1, \\ldots, x_n$，似然函数是它们各自PDF的乘积：\n    $L(\\mu, \\sigma^2) = \\prod_{i=1}^n f(x_i|\\mu, \\sigma^2) = \\left( \\frac{1}{2\\pi\\sigma^2} \\right)^{n/2} \\exp\\left( -\\frac{1}{2\\sigma^2}\\sum_{i=1}^n (x_i-\\mu)^2 \\right)$\n\n2.  **构建对数似然函数 (Log-Likelihood Function)：**\n    为了简化计算，通常取似然函数的自然对数：\n    $\\ln L(\\mu, \\sigma^2) = -\\frac{n}{2}\\ln(2\\pi) - \\frac{n}{2}\\ln(\\sigma^2) - \\frac{1}{2\\sigma^2}\\sum_{i=1}^n (x_i-\\mu)^2$\n\n3.  **对

In [3]:
from pydantic import BaseModel, Field

class MathResponse(BaseModel):
    answer: str = Field(description="详细的数学推导过程")
    explanation: str = Field(description="简要说明结果的含义")

structured_model = model.with_structured_output(MathResponse)

user_question = "请推导Normal分布的最大似然估计(MLE)公式"

print("使用Pydantic结构化输出:")
result = structured_model.invoke(user_question)

print("类型:", type(result))
print("答案:")
print(result.answer)  # type: ignore
print("\n说明:")
print(result.explanation)  # type: ignore

# 可以直接访问属性，也可以转换为字典
print("\n转换为字典:")
if hasattr(result, 'model_dump'):
    result_dict = result.model_dump()  # type: ignore
else:
    result_dict = result
print(result_dict)


使用Pydantic结构化输出:
类型: <class '__main__.MathResponse'>
答案:
Normal分布的概率密度函数（PDF）为：
$f(x | \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$

对于一个包含 $n$ 个独立同分布（i.i.d.）样本 $X = \{x_1, x_2, ..., x_n\}$ 的数据集，其似然函数为：
$L(\mu, \sigma^2 | X) = \prod_{i=1}^{n} f(x_i | \mu, \sigma^2) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x_i-\mu)^2}{2\sigma^2}}$

为了简化计算，我们取似然函数的自然对数，得到对数似然函数：
$ln L(\mu, \sigma^2 | X) = \sum_{i=1}^{n} ln \left( \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x_i-\mu)^2}{2\sigma^2}} \right)$
$ln L(\mu, \sigma^2 | X) = \sum_{i=1}^{n} \left( -\frac{1}{2}ln(2\pi) - \frac{1}{2}ln(\sigma^2) - \frac{(x_i-\mu)^2}{2\sigma^2} \right)$
$ln L(\mu, \sigma^2 | X) = -\frac{n}{2}ln(2\pi) - \frac{n}{2}ln(\sigma^2) - \frac{1}{2\sigma^2}\sum_{i=1}^{n}(x_i-\mu)^2$

为了找到最大似然估计值，我们对对数似然函数分别关于 $\mu$ 和 $\sigma^2$ 求偏导数，并令其等于零。

**1. 对 $\mu$ 求偏导：**
$\frac{\partial}{\partial \mu} ln L = \frac{\partial}{\partial \mu} \left( -\frac{n}{2}ln(2\pi) - \frac{n}{2}ln(\sigma

### Structured Output优点
1. 稳定易用
2. prompt与输出格式分离，方便维护
3. 自动有pydantic的类型验证

### 为什么不应该自己实现JSON模式？

虽然方法一（手动解析JSON）看起来简单直接，但实际上存在许多问题：

#### 1. 可靠性问题
- **格式不稳定**：LLM可能返回不符合JSON格式的内容
- **解析错误**：手动解析容易出现各种边界情况

#### 2. 维护成本
- **错误处理复杂**：需要处理各种解析异常
- **格式验证**：需要手动验证字段是否存在和类型是否正确
- **代码重复**：每个不同的结构都需要重新编写解析逻辑

#### 3. 缺乏类型安全
- **运行时错误**：字段访问错误只能在运行时发现
- **IDE支持差**：无法获得代码补全和类型提示
- **重构困难**：修改数据结构时容易遗漏更新

### 结论

需要结构化输出的时候，
1. 首先考虑无脑选择LangChain的with_structured_output
2. 极少数情况，考虑使用JSON mode（Google, Anthropic已经放弃支持）
3. 几乎不可能出现的情况：自己clean and parse JSON

需要调用工具的时候，参考下一节，
1. 首先无脑选择LangChain的 .bind_tools()
2. 极少数情况，使用各家API的 function calling (tool calling) 功能
3. 几乎不可能出现的情况：自己定义function calling的schema并试图parse